# Task 4: Creating an Interactive Chat Interface
To build a user-friendly interface that allows non-technical users to interact with prevouse RAG system.

In [ ]:
# installing of dependenceis
!pip install gradio pandas langchain langchain-community sentence-transformers faiss-cpu transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 794.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45

In [ ]:
import gradio as gr
import pandas as pd
from pathlib import Path
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import pickle


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload vector_store.zip
!unzip vector_store.zip -d /content/

# RAG pipelines
- this function for the retrieve relevant complaint chunks and generate an answer or
- Reuses the RAG pipeline from Task 3, taking a question, retrieving top-5 chunks from the FAISS vector store, and generating an answer using google/flan-t5-base.

In [ ]:

def run_rag_pipeline(vector_store_path='content', question=None, k=5,
                     llm_model='google/flan-t5-base'):
    """
    Run the RAG pipeline: retrieve relevant complaint chunks and generate an answer.

    Args:
        vector_store_path (str): Directory containing the FAISS vector store and embedding model
        question (str): User question to process
        k (int): Number of chunks to retrieve
        llm_model (str): Hugging Face model for text generation (default: google/flan-t5-base)

    Returns:
        dict: Contains answer and retrieved documents
    """
    # Set up paths /content/faiss_index
    VECTOR_STORE_PATH = Path(vector_store_path)
    VECTOR_STORE_FILE = '/content / faiss_index'
    EMBEDDING_MODEL_FILE = '/content / embedding_model.pkl'

    # Load embedding model
    print("Loading embedding model...")
    with open(EMBEDDING_MODEL_FILE, 'rb') as f:
        embedding_model = pickle.load(f)

    # Load vector store
    print("Loading vector store...")
    vector_store = FAISS.load_local(VECTOR_STORE_FILE, embeddings=embedding_model, allow_dangerous_deserialization=True)

    # Initialize LLM
    print(f"Loading language model ({llm_model})...")
    llm = pipeline('text2text-generation', model=llm_model, max_length=150)

    # Prompt template
    PROMPT_TEMPLATE = """You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints based only on the provided context. If the context doesn't contain enough information to answer the question, state that clearly and do not make assumptions. Provide a concise and accurate answer.

Context:
{context}

Question:
{question}

Answer:
"""

    # Embed the question
    question_embedding = embedding_model.encode([question])[0]

    # Perform similarity search
    retrieved_docs = vector_store.similarity_search_by_vector(question_embedding, k=k)

    # Combine context from retrieved documents
    context = "\n".join([f"Complaint ID {doc.metadata['complaint_id']} (Product: {doc.metadata['product']}): {doc.page_content}"
                         for doc in retrieved_docs])

    # Format prompt
    prompt = PROMPT_TEMPLATE.format(context=context, question=question)

    # Generate response
    response = llm(prompt, num_return_sequences=1)[0]['generated_text']

    # Extract answer
    answer = response.strip()

    return {
        'answer': answer,
        'retrieved_docs': retrieved_docs
    }

In [ ]:
def chat_interface(question, history):
    """
    Gradio chat interface function to handle user questions and display answers with sources.

    Args:
        question (str): User's question
        history (list): Chat history

    Returns:
        tuple: (answer, sources, updated history)
    """
    if not question:
        return "Please enter a question.", "", history

    # Run RAG pipeline
    result = run_rag_pipeline(question=question)

    # Format answer
    answer = result['answer']

    # Format sources
    sources = "\n\n".join([f"**Source {i+1} (Complaint ID {doc.metadata['complaint_id']}, Product: {doc.metadata['product']})**:\n{doc.page_content[:200]}..."
                          for i, doc in enumerate(result['retrieved_docs'][:3])])

    # Update chat history
    history.append((question, f"{answer}\n\n**Retrieved Sources**:\n{sources}"))

    return answer, sources, history


In [ ]:
def clear_conversation():
    """
    Clear the chat history.

    Returns:
        tuple: Empty history and cleared outputs
    """
    return [], "", ""

# Gradio interface
with gr.Blocks(title="CrediTrust Complaint Analyzer") as demo:
    gr.Markdown("# CrediTrust Complaint Analyzer")
    gr.Markdown("Ask questions about customer complaints from the CFPB dataset.")

    # Chatbot for conversation history
    chatbot = gr.Chatbot(label="Conversation", type='messages')

    # Question input
    question = gr.Textbox(label="Your Question", placeholder="e.g., What are common issues with credit card complaints?")

    # Buttons
    with gr.Row():
        submit_button = gr.Button("Submit")
        clear_button = gr.Button("Clear")

    # Output for answer
    answer_output = gr.Textbox(label="Answer", interactive=False)

    # Output for sources
    sources_output = gr.Textbox(label="Retrieved Sources", interactive=False)

    # Bind submit button
    submit_button.click(
        fn=chat_interface,
        inputs=[question, chatbot],
        outputs=[answer_output, sources_output, chatbot]
    )

    # Bind clear button
    clear_button.click(
        fn=clear_conversation,
        inputs=[],
        outputs=[chatbot, answer_output, sources_output]
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4048aad81a3f92b63b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading embedding model...


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2229, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1740, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^